In [19]:
!pip install pandas matplotlib seaborn reportlab openpyxl python-docx docx2pdf

In [23]:
!apt-get update && apt-get install -y libreoffice

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,715 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,211 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,909 k

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import textwrap
from datetime import datetime
import locale
import subprocess # Para ejecutar LibreOffice

# --- NUEVAS IMPORTACIONES PARA DOCX ---
from docx import Document
from docx.shared import Inches, Pt # Para tamaños de imagen y fuente
from docx.enum.text import WD_ALIGN_PARAGRAPH # Para alineación de párrafos
from docx.oxml.ns import qn # Para nombres de fuentes complejos
from docx.shared import RGBColor # Para colores de fuente

import matplotlib
matplotlib.use('Agg') # Usar el backend Agg para evitar problemas en entornos sin GUI

# --- CONFIGURACIÓN DE LOCALE PARA FECHAS EN ESPAÑOL ---
try:
    locale.setlocale(locale.LC_TIME, 'es_CO.UTF-8') # Intenta con Colombia
except locale.Error:
    try:
        locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8') # Intenta con España general
    except locale.Error:
        try:
            locale.setlocale(locale.LC_TIME, '') # Fallback al locale del sistema
            print("Advertencia: Locale 'es_CO.UTF-8' o 'es_ES.UTF-8' no encontrado. Usando locale por defecto del sistema para fechas.")
        except locale.Error:
            # Si todo falla, las fechas podrían no usar nombres de meses en español.
            print("Advertencia: No se pudo configurar el locale para formateo de fechas en español.")


# Configuración mejorada para gráficos profesionales (se mantiene para los PNG)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica', 'DejaVu Sans']
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 16
plt.rcParams['figure.figsize'] = (13, 8)

sns.set_style("ticks")

# --- RUTAS DE ARCHIVOS ---
ruta_excel = "/content/drive/MyDrive/EVALUACIÓN DOCENTE/Seguimiento a Syllabus(5001-6425).xlsx"
ruta_plantilla_docx = "/content/drive/MyDrive/EVALUACIÓN DOCENTE/plantilla.docx" # Asegúrate que esta plantilla exista
carpeta_graficas = "/content/drive/MyDrive/EVALUACIÓN DOCENTE/graficas_docente_espacio_docx"
os.makedirs(carpeta_graficas, exist_ok=True)

# Nombre del archivo DOCX temporal y PDF final
temp_docx_output_path = "/content/drive/MyDrive/EVALUACIÓN DOCENTE/Informe_Evaluacion_Syllabus_Temp.docx"
final_pdf_output_path = "/content/drive/MyDrive/EVALUACIÓN DOCENTE/Informe_Evaluacion_Syllabus_con_Plantilla.pdf"


# Cargar el archivo Excel
try:
    df_original = pd.read_excel(ruta_excel)
except FileNotFoundError:
    print(f"Error: El archivo Excel no se encontró en la ruta especificada: {ruta_excel}")
    exit()
except Exception as e:
    print(f"Error al leer el archivo Excel: {e}")
    exit()


# Filtrar el DataFrame principal una sola vez
df_filtro_principal = df_original[df_original['Corte Academico'] == 'Primer Corte 2025 - 1']
df = df_filtro_principal.copy() # Usar 'df' como el DataFrame filtrado consistentemente

if df.empty:
    print("El DataFrame está vacío después de filtrar por 'Primer Corte 2025 - 1'. No se generará ningún informe.")
    exit()

# Preguntas a evaluar, etiquetas cortas, mapeo y colores
preguntas = [
    "Las temáticas desarrolladas corresponden a las registradas en el syllabus.",
    "Las temáticas o contenidos se desarrollan acorde a los tiempos planeados en el syllabus",
    "Las actividades didácticas desarrolladas son acordes a la temática que se está abordado",
    "La estrategia didáctica es pertinente para el desarrollo de los resultados de aprendizaje que se pretende desarrollar.",
    "Los mecanismos (estrategias) de evaluación son pertinentes para comprobar los resultados de aprendizaje que se esperan desarrollar en el espacio académico.",
    "Se implementan diferentes metodologías de evaluación"
]
preguntas_cortas = [
    "Temáticas\ncorresponden\nal syllabus", "Temáticas siguen\ntiempos\nplaneados",
    "Actividades\nacordes a\nla temática", "Estrategia\ndidáctica\npertinente",
    "Mecanismos de\nevaluación\npertinentes", "Diversidad de\nmetodologías\nde evaluación"
]
mapeo_preguntas = dict(zip(preguntas, preguntas_cortas))
orden_etiquetas = ['Siempre', 'Muy a menudo', 'A menudo', 'Raras veces', 'Casi Nunca']
colores_palette = ["#1a73e8", "#4285f4", "#fbbc04", "#ea4335", "#c53929"]
paleta_colores = dict(zip(orden_etiquetas, colores_palette))
docentes = df['Docente del Espacio Académico'].dropna().unique()


# --- INICIALIZAR DOCUMENTO DOCX DESDE PLANTILLA ---
try:
    document = Document(ruta_plantilla_docx)
except Exception as e:
    print(f"Error al cargar la plantilla DOCX desde: {ruta_plantilla_docx}. Asegúrate que el archivo existe.")
    print(f"Detalle del error: {e}")
    exit()

# --- FUNCIONES AUXILIARES PARA AÑADIR TEXTO CON ESTILO ---
def add_styled_paragraph(doc, text, font_name='Helvetica', font_size=12, is_bold=False, is_italic=False, align=WD_ALIGN_PARAGRAPH.LEFT, space_before_pt=0, space_after_pt=6, color_hex=None):
    p = doc.add_paragraph()
    p.alignment = align
    p.paragraph_format.space_before = Pt(space_before_pt)
    p.paragraph_format.space_after = Pt(space_after_pt)

    run = p.add_run(text)
    try:
        run.font.name = font_name
        # Para fuentes no estándar o mayor compatibilidad:
        r = run._element
        r.rPr.rFonts.set(qn('w:eastAsia'), font_name)
        r.rPr.rFonts.set(qn('w:cs'), font_name)
    except Exception as e:
        print(f"Advertencia: No se pudo establecer la fuente '{font_name}'. Error: {e}")

    run.font.size = Pt(font_size)
    run.bold = is_bold
    run.italic = is_italic
    if color_hex:
        try:
            run.font.color.rgb = RGBColor.from_string(color_hex)
        except ValueError:
            print(f"Advertencia: Color hexadecimal '{color_hex}' no válido.")
    return p

# --- DEFINICIÓN DE ESTILOS PARA DOCX (simulando los de reportlab) ---
def add_titulo_portada(doc, text):
    add_styled_paragraph(doc, text, font_name='Helvetica', font_size=22, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=12, space_after_pt=20, color_hex='003366')

def add_subtitulo_institucion_portada(doc, text):
    add_styled_paragraph(doc, text, font_name='Helvetica', font_size=16, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_after_pt=5, color_hex='202124')

def add_texto_portada_info(doc, text):
    lines = text.split('<br/>') # Manejar múltiples líneas si se usa <br/>
    for i, line in enumerate(lines):
        space_after = 10 if i == len(lines) - 1 else 0
        add_styled_paragraph(doc, line, font_name='Helvetica', font_size=12, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=10 if i==0 else 0 , space_after_pt=space_after, color_hex='2F4F4F') # darkslategray

def add_texto_portada_elaborado(doc, text):
    add_styled_paragraph(doc, text, font_name='Helvetica', font_size=12, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=20, space_after_pt=4, color_hex='2F4F4F')

def add_texto_portada_autor(doc, text):
    add_styled_paragraph(doc, text, font_name='Helvetica', font_size=12, align=WD_ALIGN_PARAGRAPH.CENTER, space_after_pt=2, color_hex='2F4F4F')

def add_texto_portada_cargo(doc, text):
    add_styled_paragraph(doc, text, font_name='Helvetica', font_size=11, is_italic=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_after_pt=15, color_hex='2F4F4F')

def add_texto_portada_fecha_gen(doc, text):
    add_styled_paragraph(doc, text, font_name='Helvetica', font_size=9, is_italic=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=5, space_after_pt=4, color_hex='808080') # grey

def add_titulo_seccion(doc, text):
    add_styled_paragraph(doc, text, font_name='Helvetica', font_size=18, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_after_pt=12, color_hex='1a73e8')

def add_subtitulo_seccion(doc, text):
    add_styled_paragraph(doc, text, font_name='Helvetica', font_size=14, is_bold=True, space_after_pt=6, color_hex='202124')

def add_subsubtitulo_seccion(doc, text):
    add_styled_paragraph(doc, text, font_name='Helvetica', font_size=12, is_bold=True, space_after_pt=6, color_hex='5f6368') # Hice este negrita para destacar

def add_texto_normal(doc, text):
    add_styled_paragraph(doc, text, font_name='Helvetica', font_size=10, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=6)

def add_spacer_docx(doc, points=12):
    p = doc.add_paragraph()
    p.paragraph_format.space_before = Pt(0)
    p.paragraph_format.space_after = Pt(points)
    p.text = "" # Añadir texto vacío para que el espaciador tenga efecto visible

# --- INFORMACIÓN PARA LA PORTADA ---
corte_academico_valor = "No especificado"
if not df.empty and 'Corte Academico' in df.columns:
    valores_corte = df['Corte Academico'].unique()
    if len(valores_corte) > 0:
        corte_academico_valor = str(valores_corte[0])

fecha_min_str = "No disponible"
fecha_max_str = "No disponible"
if 'Hora de inicio' in df.columns and not df.empty:
    df_temp_fechas = df.copy()
    df_temp_fechas['Hora de inicio'] = pd.to_datetime(df_temp_fechas['Hora de inicio'], errors='coerce')
    fechas_validas = df_temp_fechas['Hora de inicio'].dropna()
    if not fechas_validas.empty:
        fecha_inicio_min = fechas_validas.min()
        fecha_inicio_max = fechas_validas.max()
        try:
            fecha_min_str = fecha_inicio_min.strftime('%d de %B de %Y')
            fecha_max_str = fecha_inicio_max.strftime('%d de %B de %Y')
        except ValueError: # Fallback si el locale no pudo formatear con nombres de mes
            fecha_min_str = fecha_inicio_min.strftime('%Y-%m-%d')
            fecha_max_str = fecha_inicio_max.strftime('%Y-%m-%d')
            print("Advertencia: Falló el formateo de fecha con nombre de mes para el rango, usando formato YYYY-MM-DD.")
    else:
        print("Advertencia: No hay fechas válidas en la columna 'Hora de inicio' después de la conversión.")
else:
    print("Advertencia: La columna 'Hora de inicio' no se encuentra en el DataFrame o el DataFrame está vacío.")

fecha_generacion_informe_str = "No disponible"
try:
    fecha_generacion_informe_str = datetime.now().strftime('%d de %B de %Y')
except ValueError: # Fallback si el locale no pudo formatear con nombres de mes
    fecha_generacion_informe_str = datetime.now().strftime('%Y-%m-%d')
    print("Advertencia: Falló el formateo de fecha de generación con nombre de mes, usando formato YYYY-MM-DD.")


# --- CONSTRUCCIÓN DE LA PORTADA en DOCX ---
add_spacer_docx(document, 50) # Espacio al inicio de la página
add_titulo_portada(document, "INFORME DE EVALUACIÓN DE SYLLABUS")
add_spacer_docx(document, 30)

add_subtitulo_institucion_portada(document, "Facultad de Ingeniería Industrial")
add_subtitulo_institucion_portada(document, "Seccional Villavicencio")
add_subtitulo_institucion_portada(document, "Universidad Santo Tomás")
add_spacer_docx(document, 40)

add_texto_portada_info(document, f"Periodo Analizado: {corte_academico_valor}")
add_texto_portada_info(document, f"La información corresponde a la recopilación de datos:<br/>Desde el {fecha_min_str} hasta el {fecha_max_str}")
add_spacer_docx(document, 40)

add_texto_portada_elaborado(document, "Elaborado por:")
add_texto_portada_autor(document, "Adriana Amelia Cespedes Orjuela") # Nombre del autor
add_texto_portada_cargo(document, "Líder de Currículo de la Facultad") # Cargo del autor
add_spacer_docx(document, 30)

add_texto_portada_fecha_gen(document, f"Fecha de generación del informe: {fecha_generacion_informe_str}")
add_texto_portada_fecha_gen(document, f"Generado en Villavicencio, Meta, Colombia.")

document.add_page_break() # Fin de la portada

# --- INICIO DEL CONTENIDO DEL INFORME ---

# --- 1. ANÁLISIS GENERAL DE LA BASE DE DATOS ---
add_titulo_seccion(document, "EVALUACIÓN GENERAL DE SYLLABUS EN LA FACULTAD")
add_spacer_docx(document, 10)

total_respuestas_dataset = 0
if preguntas and not df.empty and preguntas[0] in df.columns:
    total_respuestas_dataset = df[preguntas[0]].count()
add_texto_normal(document, f"Total de respuestas en el corte: {total_respuestas_dataset}")
add_spacer_docx(document, 10)

# Gráfico general de todas las preguntas
datos_generales_plot = []
if preguntas and not df.empty:
    for pregunta in preguntas:
        if pregunta not in df.columns:
            print(f"Advertencia: La columna de pregunta '{pregunta}' no se encuentra en el DataFrame general.")
            continue
        df_pregunta_valida = df[df[pregunta].notna()]
        total_respuestas_pregunta_general = len(df_pregunta_valida)
        if total_respuestas_pregunta_general > 0:
            conteo_general = df_pregunta_valida[pregunta].value_counts().reindex(orden_etiquetas, fill_value=0)
            porcentajes_general = (conteo_general / total_respuestas_pregunta_general) * 100
            for respuesta, porcentaje in porcentajes_general.items():
                datos_generales_plot.append({
                    'Pregunta': pregunta, 'Pregunta_Abreviada': mapeo_preguntas.get(pregunta, pregunta),
                    'Respuesta': respuesta, 'Porcentaje': porcentaje,
                })
if datos_generales_plot:
    df_plot_general = pd.DataFrame(datos_generales_plot)
    fig_gen, ax_gen = plt.subplots(figsize=(10, 6.5)) # Ajustado para mejor visualización en DOCX
    fig_gen.set_facecolor('white'); ax_gen.set_facecolor('#f8f9fa')
    sns.barplot(
        data=df_plot_general, x='Pregunta_Abreviada', y='Porcentaje', hue='Respuesta',
        palette=paleta_colores, order=[mapeo_preguntas.get(p, p) for p in preguntas if p in mapeo_preguntas],
        hue_order=orden_etiquetas, ax=ax_gen, width=0.7, saturation=0.85
    )
    ax_gen.set_title('Distribución General de Respuestas por Criterio', fontsize=15, fontweight='bold', pad=15)
    ax_gen.set_xlabel('Criterios de Evaluación', fontsize=12, labelpad=10)
    ax_gen.set_ylabel('Porcentaje (%)', fontsize=12, labelpad=10)
    ax_gen.spines['top'].set_visible(False); ax_gen.spines['right'].set_visible(False)
    ax_gen.spines['left'].set_linewidth(0.5); ax_gen.spines['bottom'].set_linewidth(0.5)
    ax_gen.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#dddddd')
    legend_gen = ax_gen.legend(
        title='Respuesta', bbox_to_anchor=(1.02, 0.5), loc='center left',
        frameon=True, framealpha=0.95, edgecolor='#dddddd'
    )
    if legend_gen:
        legend_gen.get_title().set_fontsize(11); legend_gen.get_title().set_fontweight('bold')
    ax_gen.set_yticks(range(0, 101, 20)); ax_gen.set_yticklabels([f'{x}%' for x in range(0, 101, 20)])
    plt.tight_layout(rect=[0, 0, 0.83, 1]) # Ajustar rect para que la leyenda no se corte
    ruta_imagen_general = os.path.join(carpeta_graficas, "grafico_distribucion_general.png")
    plt.savefig(ruta_imagen_general, dpi=300, bbox_inches='tight', format='png'); plt.close(fig_gen)

    try:
        document.add_picture(ruta_imagen_general, width=Inches(6.5)) # Ajusta el ancho según necesites
    except Exception as e:
        print(f"Error al añadir imagen general al DOCX: {e}")
    add_spacer_docx(document, 15)
else:
    add_texto_normal(document, "No hay datos suficientes para generar el gráfico de distribución general.")

document.add_page_break()

# --- 2. ANÁLISIS AGREGADO POR DOCENTE ---
add_titulo_seccion(document, "ANÁLISIS GENERAL DE SYLLABUS POR DOCENTE")
add_spacer_docx(document, 10)

for docente_agg_idx, docente_agg in enumerate(docentes):
    add_subtitulo_seccion(document, f"Docente: {docente_agg}") # Título para cada docente
    add_spacer_docx(document, 6)

    df_docente_agg = df[df['Docente del Espacio Académico'] == docente_agg]
    if df_docente_agg.empty:
        add_texto_normal(document, "No hay datos para este docente en el corte actual.")
        if docente_agg_idx < len(docentes) -1 : document.add_page_break()
        continue

    datos_docente_agg_plot = []
    n_total_respuestas_docente_agg = 0
    if preguntas and preguntas[0] in df_docente_agg.columns:
        n_total_respuestas_docente_agg = df_docente_agg[preguntas[0]].count()

    for pregunta in preguntas:
        if pregunta not in df_docente_agg.columns:
            print(f"Advertencia: La columna '{pregunta}' no existe para el docente {docente_agg}.")
            continue
        df_docente_pregunta_valida = df_docente_agg[df_docente_agg[pregunta].notna()]
        total_respuestas_docente_pregunta = len(df_docente_pregunta_valida)

        if total_respuestas_docente_pregunta > 0:
            conteo_docente_agg = df_docente_pregunta_valida[pregunta].value_counts().reindex(orden_etiquetas, fill_value=0)
            porcentajes_docente_agg = (conteo_docente_agg / total_respuestas_docente_pregunta) * 100
            for respuesta, porcentaje in porcentajes_docente_agg.items():
                datos_docente_agg_plot.append({
                    'Pregunta': pregunta, 'Pregunta_Abreviada': mapeo_preguntas.get(pregunta, pregunta),
                    'Respuesta': respuesta, 'Porcentaje': porcentaje,
                })

    if datos_docente_agg_plot:
        df_plot_docente_agg = pd.DataFrame(datos_docente_agg_plot)
        fig_doc_agg, ax_doc_agg = plt.subplots(figsize=(10, 6.5))
        fig_doc_agg.set_facecolor('white'); ax_doc_agg.set_facecolor('#f8f9fa')
        sns.barplot(
            data=df_plot_docente_agg, x='Pregunta_Abreviada', y='Porcentaje', hue='Respuesta',
            palette=paleta_colores, order=[mapeo_preguntas.get(p,p) for p in preguntas if p in mapeo_preguntas],
            hue_order=orden_etiquetas, ax=ax_doc_agg, width=0.7, saturation=0.85
        )
        # El título del gráfico ya no incluye el nombre del docente, se puso como subtítulo de sección
        ax_doc_agg.set_title(f'Distribución de Respuestas Agregadas', fontsize=15, fontweight='bold', pad=15)
        ax_doc_agg.set_xlabel('Criterios de Evaluación', fontsize=12, labelpad=10)
        ax_doc_agg.set_ylabel('Porcentaje (%)', fontsize=12, labelpad=10)
        ax_doc_agg.spines['top'].set_visible(False); ax_doc_agg.spines['right'].set_visible(False)
        ax_doc_agg.spines['left'].set_linewidth(0.5); ax_doc_agg.spines['bottom'].set_linewidth(0.5)
        ax_doc_agg.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#dddddd')
        legend_doc_agg = ax_doc_agg.legend(
            title='Respuesta', bbox_to_anchor=(1.02, 0.5), loc='center left',
            frameon=True, framealpha=0.95, edgecolor='#dddddd'
        )
        if legend_doc_agg:
            legend_doc_agg.get_title().set_fontsize(11); legend_doc_agg.get_title().set_fontweight('bold')
        ax_doc_agg.set_yticks(range(0, 101, 20)); ax_doc_agg.set_yticklabels([f'{x}%' for x in range(0, 101, 20)])

        if n_total_respuestas_docente_agg > 0:
            texto_resp_doc = f'Total de respuestas consolidadas para el docente: {n_total_respuestas_docente_agg}'
            # Añadir este texto al DOCX en lugar de a la figura
            add_texto_normal(document, texto_resp_doc)
            add_spacer_docx(document, 6)

        plt.tight_layout(rect=[0, 0.03, 0.83, 1]) # Ajustar rect
        filename_doc_agg = f"agregado_{str(docente_agg)[:30]}".replace('/', '-').replace(' ', '_').replace(',', '')
        ruta_imagen_doc_agg = os.path.join(carpeta_graficas, f"{filename_doc_agg}.png")
        plt.savefig(ruta_imagen_doc_agg, dpi=300, bbox_inches='tight', format='png'); plt.close(fig_doc_agg)

        try:
            document.add_picture(ruta_imagen_doc_agg, width=Inches(6.5))
        except Exception as e:
            print(f"Error al añadir imagen agregada de docente al DOCX: {e}")
        add_spacer_docx(document, 15)
    else:
        add_texto_normal(document, "No hay datos suficientes para generar el gráfico agregado para este docente.")

    if docente_agg_idx < len(docentes) -1 : document.add_page_break()


# --- 3. INFORMES DETALLADOS POR DOCENTE Y ESPACIO ACADÉMICO ---
add_titulo_seccion(document, "EVALUACIÓN SYLLABUS DETALLADA POR DOCENTE Y ESPACIO ACADÉMICO")
add_spacer_docx(document, 10) # Un poco más de espacio antes de empezar con los docentes

for docente_idx, docente in enumerate(docentes):
    add_subtitulo_seccion(document, f"Docente: {docente}")
    add_spacer_docx(document, 10)

    espacios_docente = df[df['Docente del Espacio Académico'] == docente]['Espacio Académico a evaluar'].dropna().unique()

    if not espacios_docente.any():
        add_texto_normal(document, "No se encontraron espacios académicos para este docente en el corte actual.")
        if docente_idx < len(docentes) -1 : document.add_page_break()
        continue

    for espacio_idx, espacio in enumerate(espacios_docente):
        add_subsubtitulo_seccion(document, f"Espacio Académico: {espacio}") # Usando subsubtitulo
        add_spacer_docx(document, 6)

        df_filtrado = df[(df['Docente del Espacio Académico'] == docente) & (df['Espacio Académico a evaluar'] == espacio)]
        if df_filtrado.empty:
            add_texto_normal(document, "No hay datos para este espacio académico.")
            if espacio_idx < len(espacios_docente) -1 : add_spacer_docx(document, 20) # Espacio antes del siguiente espacio
            continue

        datos_espacio = []
        n_respuestas_espacio = 0
        if preguntas and preguntas[0] in df_filtrado.columns:
             n_respuestas_espacio = df_filtrado[preguntas[0]].count()

        for pregunta in preguntas:
            if pregunta not in df_filtrado.columns:
                print(f"Advertencia: La columna '{pregunta}' no existe para el docente {docente} en el espacio {espacio}.")
                continue
            df_filtrado_pregunta_valida = df_filtrado[df_filtrado[pregunta].notna()]
            total_respuestas_pregunta_espacio = len(df_filtrado_pregunta_valida)

            if total_respuestas_pregunta_espacio > 0:
                conteo_espacio = df_filtrado_pregunta_valida[pregunta].value_counts().reindex(orden_etiquetas, fill_value=0)
                porcentajes_espacio = (conteo_espacio / total_respuestas_pregunta_espacio) * 100
                for respuesta, porcentaje in porcentajes_espacio.items():
                    datos_espacio.append({
                        'Pregunta': pregunta, 'Pregunta_Abreviada': mapeo_preguntas.get(pregunta, pregunta),
                        'Respuesta': respuesta, 'Porcentaje': porcentaje, 'Frecuencia': conteo_espacio[respuesta]
                    })

        if datos_espacio:
            df_plot_espacio = pd.DataFrame(datos_espacio)
            fig_esp, ax_esp = plt.subplots(figsize=(10, 6.5))
            fig_esp.set_facecolor('white'); ax_esp.set_facecolor('#f8f9fa')
            sns.barplot(
                data=df_plot_espacio, x='Pregunta_Abreviada', y='Porcentaje', hue='Respuesta',
                palette=paleta_colores, order=[mapeo_preguntas.get(p,p) for p in preguntas if p in mapeo_preguntas],
                hue_order=orden_etiquetas, ax=ax_esp, width=0.7, saturation=0.85
            )
            # El título del gráfico ya no incluye el espacio, se puso como subsubtítulo de sección
            ax_esp.set_title(f'Distribución de Respuestas', fontsize=15, fontweight='bold', pad=15)
            ax_esp.set_xlabel('Criterios de Evaluación', fontsize=12, labelpad=10)
            ax_esp.set_ylabel('Porcentaje (%)', fontsize=12, labelpad=10)
            ax_esp.spines['top'].set_visible(False); ax_esp.spines['right'].set_visible(False)
            ax_esp.spines['left'].set_linewidth(0.5); ax_esp.spines['bottom'].set_linewidth(0.5)
            ax_esp.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#dddddd')
            legend_esp = ax_esp.legend(
                title='Respuesta', bbox_to_anchor=(1.02, 0.5), loc='center left',
                frameon=True, framealpha=0.95, edgecolor='#dddddd'
            )
            if legend_esp:
                legend_esp.get_title().set_fontsize(11); legend_esp.get_title().set_fontweight('bold')
            ax_esp.set_yticks(range(0, 101, 20)); ax_esp.set_yticklabels([f'{x}%' for x in range(0, 101, 20)])

            resumen_texto_espacio = f"Total de respuestas para este espacio: {n_respuestas_espacio}"
            add_texto_normal(document, resumen_texto_espacio)
            add_spacer_docx(document, 6)

            plt.tight_layout(rect=[0, 0, 0.83, 1]) # Ajustar rect

            filename_espacio = f"{str(docente)[:20]}_{str(espacio)[:20]}".replace('/', '-').replace(' ', '_').replace(',', '')
            ruta_imagen_espacio = os.path.join(carpeta_graficas, f"{filename_espacio}.png")
            plt.savefig(ruta_imagen_espacio, dpi=300, bbox_inches='tight', format='png'); plt.close(fig_esp)

            try:
                document.add_picture(ruta_imagen_espacio, width=Inches(6.5))
            except Exception as e:
                print(f"Error al añadir imagen de espacio académico al DOCX: {e}")
            add_spacer_docx(document, 15)
        else:
            add_texto_normal(document, "No hay datos suficientes para generar el gráfico para este espacio académico.")

        if espacio_idx < len(espacios_docente) -1 : add_spacer_docx(document, 20) # Espacio antes del siguiente espacio del mismo docente

    if docente_idx < len(docentes) -1 : document.add_page_break() # Salto de página después de cada docente, excepto el último


# --- GUARDAR EL DOCUMENTO DOCX Y CONVERTIR A PDF USANDO LIBREOFFICE ---
try:
    document.save(temp_docx_output_path)
    print(f"Informe temporal DOCX guardado en: {temp_docx_output_path}")

    # --- Convertir el DOCX a PDF usando LibreOffice ---
    print(f"Intentando convertir '{temp_docx_output_path}' a PDF usando LibreOffice...")

    output_dir_for_pdf = os.path.dirname(final_pdf_output_path)
    if not output_dir_for_pdf:
        output_dir_for_pdf = "."
    else:
        os.makedirs(output_dir_for_pdf, exist_ok=True)

    # Intentar con 'soffice' primero, luego con 'libreoffice'
    libreoffice_executables = ['soffice', 'libreoffice']
    process_result = None

    for executable in libreoffice_executables:
        command = [
            executable,
            '--headless',
            '--convert-to', 'pdf',
            '--outdir', output_dir_for_pdf,
            temp_docx_output_path
        ]
        try:
            print(f"Intentando con el comando: {' '.join(command)}")
            process_result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=False, timeout=120) # Timeout de 2 minutos
            if process_result.returncode == 0:
                print(f"Comando '{executable}' ejecutado exitosamente.")
                break # Salir del bucle si el comando tuvo éxito
            else:
                print(f"Comando '{executable}' falló con código de retorno: {process_result.returncode}")
                print(f"Stderr: {process_result.stderr.decode('utf-8', errors='ignore')}")
        except FileNotFoundError:
            print(f"Error: El ejecutable '{executable}' no fue encontrado.")
            process_result = None # Asegurarse que process_result no tenga un valor de un intento fallido
        except subprocess.TimeoutExpired:
            print(f"Error: El comando '{executable}' excedió el tiempo límite de 120 segundos.")
            process_result = None
            # Continuar al siguiente ejecutable si hay uno
        except Exception as e:
            print(f"Error inesperado al ejecutar '{executable}': {e}")
            process_result = None


    base_name_temp_docx = os.path.basename(temp_docx_output_path)
    pdf_name_generated_by_libreoffice = os.path.splitext(base_name_temp_docx)[0] + ".pdf"
    path_pdf_generated_by_libreoffice = os.path.join(output_dir_for_pdf, pdf_name_generated_by_libreoffice)

    if process_result and process_result.returncode == 0 and os.path.exists(path_pdf_generated_by_libreoffice):
        if path_pdf_generated_by_libreoffice != final_pdf_output_path:
            try:
                os.rename(path_pdf_generated_by_libreoffice, final_pdf_output_path)
                print(f"Informe PDF final con plantilla creado en: {final_pdf_output_path}")
            except OSError as e:
                print(f"Error al renombrar el archivo PDF: {e}")
                print(f"El PDF generado por LibreOffice se encuentra en: {path_pdf_generated_by_libreoffice}")
        else:
            print(f"Informe PDF final con plantilla creado en: {final_pdf_output_path}")

        # Opcional: eliminar el archivo DOCX temporal
        # try:
        #     if os.path.exists(temp_docx_output_path):
        #         os.remove(temp_docx_output_path)
        #         print(f"Archivo temporal DOCX eliminado: {temp_docx_output_path}")
        # except OSError as e:
        #     print(f"Advertencia: No se pudo eliminar el archivo DOCX temporal. {e}")

    else:
        print("-" * 50)
        print("ERROR CRÍTICO: La conversión de DOCX a PDF con LibreOffice falló después de intentar con todos los ejecutables.")
        if process_result:
            print(f"Último código de retorno de LibreOffice: {process_result.returncode}")
            print(f"Última salida estándar (stdout) de LibreOffice: {process_result.stdout.decode('utf-8', errors='ignore')}")
            print(f"Última salida de error (stderr) de LibreOffice: {process_result.stderr.decode('utf-8', errors='ignore')}")

        print("\nPOSIBLES SOLUCIONES:")
        print("1. Asegúrate de que LibreOffice esté instalado correctamente en tu sistema.")
        print("   En Google Colab, puedes instalarlo ejecutando la siguiente celda de código:")
        print("   !apt-get update && apt-get install -y libreoffice")
        print("2. Verifica que los ejecutables 'soffice' o 'libreoffice' estén en el PATH del sistema.")
        print("3. Revisa los mensajes de error de stderr de LibreOffice de arriba para más detalles.")
        if not os.path.exists(path_pdf_generated_by_libreoffice) and process_result and process_result.returncode == 0 :
             print(f"El archivo PDF esperado ({path_pdf_generated_by_libreoffice}) no fue encontrado, aunque el proceso indicó éxito.")
        elif not os.path.exists(path_pdf_generated_by_libreoffice):
             print(f"El archivo PDF esperado ({path_pdf_generated_by_libreoffice}) no fue encontrado.")
        print("-" * 50)


    print("Proceso completado.")

except Exception as e:
    print(f"Error general en el proceso de guardado o conversión: {e}")
    import traceback
    traceback.print_exc()

Advertencia: Locale 'es_CO.UTF-8' o 'es_ES.UTF-8' no encontrado. Usando locale por defecto del sistema para fechas.
Informe temporal DOCX guardado en: /content/drive/MyDrive/EVALUACIÓN DOCENTE/Informe_Evaluacion_Syllabus_Temp.docx
Intentando convertir '/content/drive/MyDrive/EVALUACIÓN DOCENTE/Informe_Evaluacion_Syllabus_Temp.docx' a PDF usando LibreOffice...
Intentando con el comando: soffice --headless --convert-to pdf --outdir /content/drive/MyDrive/EVALUACIÓN DOCENTE /content/drive/MyDrive/EVALUACIÓN DOCENTE/Informe_Evaluacion_Syllabus_Temp.docx
Comando 'soffice' ejecutado exitosamente.
Informe PDF final con plantilla creado en: /content/drive/MyDrive/EVALUACIÓN DOCENTE/Informe_Evaluacion_Syllabus_con_Plantilla.pdf
Proceso completado.


In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import textwrap
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image, Paragraph, PageBreak, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib import colors # Import the colors module
from reportlab.lib.enums import TA_CENTER, TA_JUSTIFY
import matplotlib
matplotlib.use('Agg')  # Usar el backend Agg para evitar problemas en entornos sin GUI
from datetime import datetime # Para la fecha de generación
import locale # Para formatear fechas en español

# --- CONFIGURACIÓN DE LOCALE PARA FECHAS EN ESPAÑOL ---
try:
    locale.setlocale(locale.LC_TIME, 'es_CO.UTF-8') # Intenta con Colombia
except locale.Error:
    try:
        locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8') # Intenta con España general
    except locale.Error:
        try:
            locale.setlocale(locale.LC_TIME, '') # Fallback al locale del sistema
            print("Advertencia: Locale 'es_CO.UTF-8' o 'es_ES.UTF-8' no encontrado. Usando locale por defecto del sistema para fechas.")
        except locale.Error:
            # Si todo falla, las fechas podrían no usar nombres de meses en español.
            print("Advertencia: No se pudo configurar el locale para formateo de fechas en español.")


# Configuración mejorada para gráficos profesionales
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica', 'DejaVu Sans']
# ... (resto de plt.rcParams)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 16
plt.rcParams['figure.figsize'] = (13, 8)


# Establecer un estilo minimalista pero profesional
sns.set_style("ticks")  # Estilo más limpio con marcas en los ejes

# Cargar el archivo
try:
    df_original = pd.read_excel("/content/drive/MyDrive/EVALUACIÓN DOCENTE/Seguimiento a Syllabus(5001-6425).xlsx")
except FileNotFoundError:
    print("Error: El archivo Excel no se encontró en la ruta especificada.")
    exit()

# Filtrar el DataFrame principal una sola vez
df_filtro_principal = df_original[df_original['Corte Academico'] == 'Primer Corte 2025 - 1']
df = df_filtro_principal.copy() # Usar 'df' como el DataFrame filtrado consistentemente

if df.empty:
    print("El DataFrame está vacío después de filtrar por 'Primer Corte 2025 - 1'. No se generará ningún informe.")
    exit()

# Crear carpetas de salida
carpeta_graficas = "/content/drive/MyDrive/EVALUACIÓN DOCENTE/graficas_docente_espacio"
os.makedirs(carpeta_graficas, exist_ok=True)

# Preguntas a evaluar, etiquetas cortas, mapeo
preguntas = [
    "Las temáticas desarrolladas corresponden a las registradas en el syllabus.",
    "Las temáticas o contenidos se desarrollan acorde a los tiempos planeados en el syllabus",
    "Las actividades didácticas desarrolladas son acordes a la temática que se está abordado",
    "La estrategia didáctica es pertinente para el desarrollo de los resultados de aprendizaje que se pretende desarrollar.",
    "Los mecanismos (estrategias) de evaluación son pertinentes para comprobar los resultados de aprendizaje que se esperan desarrollar en el espacio académico.",
    "Se implementan diferentes metodologías de evaluación"
]
preguntas_cortas = [
    "Temáticas\ncorresponden\nal syllabus", "Temáticas siguen\ntiempos\nplaneados",
    "Actividades\nacordes a\nla temática", "Estrategia\ndidáctica\npertinente",
    "Mecanismos de\nevaluación\npertinentes", "Diversidad de\nmetodologías\nde evaluación"
]
mapeo_preguntas = dict(zip(preguntas, preguntas_cortas))
orden_etiquetas = ['Siempre', 'Muy a menudo', 'A menudo', 'Raras veces', 'Casi Nunca']
colores_palette = ["#1a73e8", "#4285f4", "#fbbc04", "#ea4335", "#c53929"]
paleta_colores = dict(zip(orden_etiquetas, colores_palette))
docentes = df['Docente del Espacio Académico'].dropna().unique()

# PDF setup
informe_completo_pdf = "/content/drive/MyDrive/EVALUACIÓN DOCENTE/Informe_Evaluacion_Syllabus.pdf" # Nuevo nombre para evitar sobreescribir
doc = SimpleDocTemplate(informe_completo_pdf, pagesize=letter)
elementos = []

# Estilos para el PDF
estilos = getSampleStyleSheet()

# Estilos generales existentes (modificados o usados como base para la portada)
estilo_titulo_original = ParagraphStyle( # Renombrado para claridad, usado para títulos de sección
    'TituloSeccion',
    parent=estilos['Heading1'],
    alignment=TA_CENTER,
    fontName='Helvetica-Bold',
    fontSize=18,
    spaceAfter=12,
    textColor=colors.HexColor('#1a73e8')
)
estilo_subtitulo = ParagraphStyle(
    'Subtitulo', parent=estilos['Heading2'], fontName='Helvetica-Bold', fontSize=14,
    spaceAfter=6, textColor=colors.HexColor('#202124')
)
estilo_subsubtitulo = ParagraphStyle(
    'Subsubtitulo', parent=estilos['Heading3'], fontName='Helvetica', fontSize=12,
    spaceAfter=6, textColor=colors.HexColor('#5f6368')
)
estilo_normal = ParagraphStyle(
    'TextoNormal', parent=estilos['Normal'], fontName='Helvetica', fontSize=10,
    alignment=TA_JUSTIFY, spaceAfter=6
)

# --- NUEVOS ESTILOS PARA LA PORTADA ---
estilo_titulo_portada = ParagraphStyle(
    'TituloPortada', parent=estilos['Heading1'], alignment=TA_CENTER,
    fontName='Helvetica-Bold', fontSize=22, spaceBefore=12, spaceAfter=20,
    textColor=colors.HexColor('#003366') # Un azul más oscuro para el título de portada
)
estilo_subtitulo_institucion_portada = ParagraphStyle(
    'SubtituloInstitucionPortada', parent=estilos['Heading2'], alignment=TA_CENTER,
    fontName='Helvetica-Bold', fontSize=16, spaceAfter=5, leading=20,
    textColor=colors.HexColor('#202124')
)
estilo_texto_portada_info = ParagraphStyle(
    'TextoPortadaInfo', parent=estilos['Normal'], alignment=TA_CENTER,
    fontName='Helvetica', fontSize=12, leading=18, # Buen interlineado
    spaceBefore=10, spaceAfter=10, textColor=colors.darkslategray
)
estilo_texto_portada_elaborado = ParagraphStyle(
    'TextoPortadaElaborado', parent=estilo_texto_portada_info,
    fontName='Helvetica-Bold', spaceBefore=20, spaceAfter=4,
    textColor=colors.darkslategray
)
estilo_texto_portada_autor = ParagraphStyle(
    'TextoPortadaAutor', parent=estilo_texto_portada_info,
    fontName='Helvetica', spaceBefore=0, spaceAfter=2,
    textColor=colors.darkslategray
)
estilo_texto_portada_cargo = ParagraphStyle(
    'TextoPortadaCargo', parent=estilo_texto_portada_info,
    fontName='Helvetica-Oblique', fontSize=11, spaceBefore=0, spaceAfter=15,
    textColor=colors.darkslategray
)
estilo_texto_portada_fecha_gen = ParagraphStyle(
    'TextoPortadaFechaGen', parent=estilos['Normal'], alignment=TA_CENTER,
    fontName='Helvetica-Oblique', fontSize=9, textColor=colors.grey,
    spaceBefore=30, spaceAfter=4
)

# --- INFORMACIÓN PARA LA PORTADA ---
corte_academico_valor = "No especificado"
if not df.empty and 'Corte Academico' in df.columns:
    valores_corte = df['Corte Academico'].unique()
    if len(valores_corte) > 0:
        corte_academico_valor = valores_corte[0]

fecha_min_str = "No disponible"
fecha_max_str = "No disponible"
if 'Hora de inicio' in df.columns and not df.empty:
    df_temp_fechas = df.copy() # Trabajar sobre una copia para evitar SettingWithCopyWarning
    df_temp_fechas['Hora de inicio'] = pd.to_datetime(df_temp_fechas['Hora de inicio'], errors='coerce')
    fechas_validas = df_temp_fechas['Hora de inicio'].dropna()
    if not fechas_validas.empty:
        fecha_inicio_min = fechas_validas.min()
        fecha_inicio_max = fechas_validas.max()
        try:
            fecha_min_str = fecha_inicio_min.strftime('%d de %B de %Y')
            fecha_max_str = fecha_inicio_max.strftime('%d de %B de %Y')
        except ValueError:
            fecha_min_str = fecha_inicio_min.strftime('%Y-%m-%d')
            fecha_max_str = fecha_inicio_max.strftime('%Y-%m-%d')
            print("Advertencia: Falló el formateo de fecha con nombre de mes, usando formato YYYY-MM-DD.")
    else:
        print("Advertencia: No hay fechas válidas en la columna 'Hora de inicio' después de la conversión.")
else:
    print("Advertencia: La columna 'Hora de inicio' no se encuentra en el DataFrame o el DataFrame está vacío.")

fecha_generacion_informe_str = "No disponible"
try:
    fecha_generacion_informe_str = datetime.now().strftime('%d de %B de %Y')
except ValueError:
    fecha_generacion_informe_str = datetime.now().strftime('%Y-%m-%d')

# --- CONSTRUCCIÓN DE LA PORTADA ---
elementos.append(Spacer(1, 50)) # Espacio al inicio de la página
elementos.append(Paragraph("INFORME DE EVALUACIÓN DE SYLLABUS", estilo_titulo_portada))
elementos.append(Spacer(1, 30))

elementos.append(Paragraph("Facultad de Ingeniería Industrial", estilo_subtitulo_institucion_portada))
elementos.append(Paragraph("Seccional Villavicencio", estilo_subtitulo_institucion_portada))
elementos.append(Paragraph("Universidad Santo Tomás", estilo_subtitulo_institucion_portada))
elementos.append(Spacer(1, 40))

elementos.append(Paragraph(f"Periodo Analizado: {corte_academico_valor}", estilo_texto_portada_info))
elementos.append(Paragraph(f"La información corresponde a la recopilación de datos:<br/>Desde el {fecha_min_str} hasta el {fecha_max_str}", estilo_texto_portada_info))
elementos.append(Spacer(1, 40))

elementos.append(Paragraph("Elaborado por:", estilo_texto_portada_elaborado))
elementos.append(Paragraph("Adriana Amelia Cespedes Orjuela", estilo_texto_portada_autor))
elementos.append(Paragraph("Líder de Currículo de la Facultad", estilo_texto_portada_cargo))
elementos.append(Spacer(1, 30))

elementos.append(Paragraph(f"Fecha de generación del informe: {fecha_generacion_informe_str}", estilo_texto_portada_fecha_gen))
elementos.append(Paragraph(f"Generado en Villavicencio, Meta, Colombia.", estilo_texto_portada_fecha_gen))

elementos.append(PageBreak()) # Fin de la portada


# --- INICIO DEL CONTENIDO DEL INFORME ---

# --- 1. ANÁLISIS GENERAL DE LA BASE DE DATOS ---
elementos.append(Paragraph("EVALUACIÓN GENERAL DE SYLLABUS EN LA FACULTAD", estilo_subtitulo)) # Usar estilo_subtitulo aquí
elementos.append(Spacer(1, 10))
# ... (resto del código para el resumen general, gráficos agregados por docente, y detalles por espacio académico sin cambios) ...
# Asegúrate de que los títulos de las siguientes secciones usen estilos como 'estilo_subtitulo' o 'estilo_subsubtitulo' según corresponda.

# 1.1. Texto con el número total de respuestas generales
total_respuestas_dataset = 0
if preguntas and not df.empty:
    total_respuestas_dataset = df[preguntas[0]].count()
elementos.append(Paragraph(f"Total de respuestas en el corte: {total_respuestas_dataset}", estilo_normal))
elementos.append(Spacer(1, 10))

# 1.2. Gráfico general de todas las preguntas
datos_generales_plot = []
if preguntas and not df.empty:
    for pregunta in preguntas:
        df_pregunta_valida = df[df[pregunta].notna()]
        total_respuestas_pregunta_general = len(df_pregunta_valida)
        if total_respuestas_pregunta_general > 0:
            conteo_general = df_pregunta_valida[pregunta].value_counts().reindex(orden_etiquetas, fill_value=0)
            porcentajes_general = (conteo_general / total_respuestas_pregunta_general) * 100
            for respuesta, porcentaje in porcentajes_general.items():
                datos_generales_plot.append({
                    'Pregunta': pregunta, 'Pregunta_Abreviada': mapeo_preguntas[pregunta],
                    'Respuesta': respuesta, 'Porcentaje': porcentaje,
                })
if datos_generales_plot:
    df_plot_general = pd.DataFrame(datos_generales_plot)
    fig_gen, ax_gen = plt.subplots(figsize=(13, 8))
    fig_gen.set_facecolor('white'); ax_gen.set_facecolor('#f8f9fa')
    sns.barplot(
        data=df_plot_general, x='Pregunta_Abreviada', y='Porcentaje', hue='Respuesta',
        palette=paleta_colores, order=[mapeo_preguntas[p] for p in preguntas],
        hue_order=orden_etiquetas, ax=ax_gen, width=0.7, saturation=0.85
    )
    ax_gen.set_title('Distribución General de Respuestas por Criterio', fontsize=15, fontweight='bold', pad=15)
    ax_gen.set_xlabel('Criterios de Evaluación', fontsize=12, labelpad=10)
    ax_gen.set_ylabel('Porcentaje (%)', fontsize=12, labelpad=10)
    ax_gen.spines['top'].set_visible(False); ax_gen.spines['right'].set_visible(False)
    ax_gen.spines['left'].set_linewidth(0.5); ax_gen.spines['bottom'].set_linewidth(0.5)
    ax_gen.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#dddddd')
    legend_gen = ax_gen.legend(
        title='Respuesta', bbox_to_anchor=(1.02, 0.5), loc='center left',
        frameon=True, framealpha=0.95, edgecolor='#dddddd'
    )
    legend_gen.get_title().set_fontsize(11); legend_gen.get_title().set_fontweight('bold')
    ax_gen.set_yticks(range(0, 101, 20)); ax_gen.set_yticklabels([f'{x}%' for x in range(0, 101, 20)])
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    ruta_imagen_general = os.path.join(carpeta_graficas, "grafico_distribucion_general.png")
    plt.savefig(ruta_imagen_general, dpi=300, bbox_inches='tight', format='png'); plt.close(fig_gen)
    elementos.append(Image(ruta_imagen_general, width=520, height=360, kind='proportional'))
    elementos.append(Spacer(1, 15))
elementos.append(PageBreak())

# --- 2. ANÁLISIS AGREGADO POR DOCENTE ---
elementos.append(Paragraph("ANÁLISIS GENERAL DE SYLLABUS POR DOCENTE", estilo_subtitulo))
elementos.append(Spacer(1, 10))
for docente_agg in docentes:
    df_docente_agg = df[df['Docente del Espacio Académico'] == docente_agg]
    if df_docente_agg.empty or not preguntas: continue
    datos_docente_agg_plot = []
    n_total_respuestas_docente_agg = df_docente_agg[preguntas[0]].count() if preguntas else 0
    for pregunta in preguntas:
        df_docente_pregunta_valida = df_docente_agg[df_docente_agg[pregunta].notna()]
        total_respuestas_docente_pregunta = len(df_docente_pregunta_valida)
        if total_respuestas_docente_pregunta > 0:
            conteo_docente_agg = df_docente_pregunta_valida[pregunta].value_counts().reindex(orden_etiquetas, fill_value=0)
            porcentajes_docente_agg = (conteo_docente_agg / total_respuestas_docente_pregunta) * 100
            for respuesta, porcentaje in porcentajes_docente_agg.items():
                datos_docente_agg_plot.append({
                    'Pregunta': pregunta, 'Pregunta_Abreviada': mapeo_preguntas[pregunta],
                    'Respuesta': respuesta, 'Porcentaje': porcentaje,
                })
    if datos_docente_agg_plot:
        df_plot_docente_agg = pd.DataFrame(datos_docente_agg_plot)
        fig_doc_agg, ax_doc_agg = plt.subplots(figsize=(13, 8))
        fig_doc_agg.set_facecolor('white'); ax_doc_agg.set_facecolor('#f8f9fa')
        sns.barplot(
            data=df_plot_docente_agg, x='Pregunta_Abreviada', y='Porcentaje', hue='Respuesta',
            palette=paleta_colores, order=[mapeo_preguntas[p] for p in preguntas],
            hue_order=orden_etiquetas, ax=ax_doc_agg, width=0.7, saturation=0.85
        )
        ax_doc_agg.set_title(f'Resultado para el docente: {textwrap.shorten(docente_agg, width=50, placeholder="...")}', fontsize=15, fontweight='bold', pad=15)
        ax_doc_agg.set_xlabel('Criterios de Evaluación', fontsize=12, labelpad=10)
        ax_doc_agg.set_ylabel('Porcentaje (%)', fontsize=12, labelpad=10)
        ax_doc_agg.spines['top'].set_visible(False); ax_doc_agg.spines['right'].set_visible(False)
        ax_doc_agg.spines['left'].set_linewidth(0.5); ax_doc_agg.spines['bottom'].set_linewidth(0.5)
        ax_doc_agg.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#dddddd')
        legend_doc_agg = ax_doc_agg.legend(
            title='Respuesta', bbox_to_anchor=(1.02, 0.5), loc='center left',
            frameon=True, framealpha=0.95, edgecolor='#dddddd'
        )
        legend_doc_agg.get_title().set_fontsize(11); legend_doc_agg.get_title().set_fontweight('bold')
        ax_doc_agg.set_yticks(range(0, 101, 20)); ax_doc_agg.set_yticklabels([f'{x}%' for x in range(0, 101, 20)])
        if n_total_respuestas_docente_agg > 0:
             texto_resp_doc = f'Total de respuestas: {n_total_respuestas_docente_agg}'
             fig_doc_agg.text(0.02, 0.01, texto_resp_doc, fontsize=9, style='italic', transform=fig_doc_agg.transFigure)
        plt.tight_layout(rect=[0, 0.03, 0.85, 1])
        filename_doc_agg = f"agregado_{docente_agg[:30]}".replace('/', '-').replace(' ', '_').replace(',', '')
        ruta_imagen_doc_agg = os.path.join(carpeta_graficas, f"{filename_doc_agg}.png")
        plt.savefig(ruta_imagen_doc_agg, dpi=300, bbox_inches='tight', format='png'); plt.close(fig_doc_agg)
        elementos.append(Paragraph(f"Docente: {docente_agg}", estilo_subsubtitulo))
        elementos.append(Spacer(1, 6))
        elementos.append(Image(ruta_imagen_doc_agg, width=520, height=360, kind='proportional'))
        elementos.append(Spacer(1, 15))
    elementos.append(PageBreak())

# --- 3. INFORMES DETALLADOS POR DOCENTE Y ESPACIO ACADÉMICO ---
elementos.append(Paragraph("EVALUACIÓN SYLLABUS DE DOCENTE SEGÚN ESPACIO ACADÉMICO", estilo_subtitulo))
elementos.append(Spacer(1, 20))
for docente in docentes:
    elementos.append(Paragraph(f"Docente: {docente}", estilo_subtitulo)) # Subtitulo para la sección detallada del docente
    elementos.append(Spacer(1, 10))
    espacios_docente = df[df['Docente del Espacio Académico'] == docente]['Espacio Académico a evaluar'].dropna().unique()
    if not espacios_docente.any():
        elementos.append(Paragraph("No se encontraron espacios académicos para este docente en el corte actual.", estilo_normal))
        elementos.append(PageBreak()); continue
    for espacio in espacios_docente:
        df_filtrado = df[(df['Docente del Espacio Académico'] == docente) & (df['Espacio Académico a evaluar'] == espacio)]
        if df_filtrado.empty or not preguntas: continue
        datos = []
        n_respuestas = df_filtrado[preguntas[0]].count() if preguntas else 0
        for pregunta in preguntas:
            df_filtrado_pregunta_valida = df_filtrado[df_filtrado[pregunta].notna()]
            total_respuestas = len(df_filtrado_pregunta_valida)
            if total_respuestas > 0:
                conteo = df_filtrado_pregunta_valida[pregunta].value_counts().reindex(orden_etiquetas, fill_value=0)
                porcentajes = (conteo / total_respuestas) * 100
                for respuesta, porcentaje in porcentajes.items():
                    datos.append({
                        'Pregunta': pregunta, 'Pregunta_Abreviada': mapeo_preguntas[pregunta],
                        'Respuesta': respuesta, 'Porcentaje': porcentaje, 'Frecuencia': conteo[respuesta]
                    })
        if datos:
            df_plot = pd.DataFrame(datos)
            fig, ax = plt.subplots(figsize=(13, 8))
            fig.set_facecolor('white'); ax.set_facecolor('#f8f9fa')
            sns.barplot(
                data=df_plot, x='Pregunta_Abreviada', y='Porcentaje', hue='Respuesta',
                palette=paleta_colores, order=[mapeo_preguntas[p] for p in preguntas],
                hue_order=orden_etiquetas, ax=ax, width=0.7, saturation=0.85
            )
            ax.set_xlabel('Criterios de Evaluación', fontsize=12, labelpad=10)
            ax.set_ylabel('Porcentaje (%)', fontsize=12, labelpad=10)
            ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
            ax.spines['left'].set_linewidth(0.5); ax.spines['bottom'].set_linewidth(0.5)
            ax.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#dddddd')
            legend = ax.legend(
                title='Frecuencia', bbox_to_anchor=(1.02, 0.5), loc='center left',
                frameon=True, framealpha=0.95, edgecolor='#dddddd'
            )
            legend.get_title().set_fontsize(11); legend.get_title().set_fontweight('bold')
            ax.set_yticks(range(0, 101, 20)); ax.set_yticklabels([f'{x}%' for x in range(0, 101, 20)])
            plt.tight_layout(rect=[0, 0, 0.85, 1])
            filename = f"{docente[:20]}_{espacio[:20]}".replace('/', '-').replace(' ', '_').replace(',', '')
            ruta_imagen = os.path.join(carpeta_graficas, f"{filename}.png")
            plt.savefig(ruta_imagen, dpi=300, bbox_inches='tight', format='png'); plt.close(fig)
            elementos.append(Paragraph(f"Espacio Académico: {espacio}", estilo_subsubtitulo))
            elementos.append(Spacer(1, 6))
            resumen_texto = f"Total de respuestas: {n_respuestas}"
            elementos.append(Paragraph(resumen_texto, estilo_normal))
            elementos.append(Spacer(1, 10))
            elementos.append(Image(ruta_imagen, width=520, height=360, kind='proportional'))
            elementos.append(Spacer(1, 15))
        elementos.append(PageBreak())

# Construir el PDF
if elementos:
    try:
        doc.build(elementos)
        print(f"Informe completo con portada creado en: {informe_completo_pdf}")
        print("Proceso completado.")
    except Exception as e:
        print(f"Error al construir el PDF: {e}")
else:
    print("No se generaron elementos para el PDF. Verifique los datos de entrada y filtros.")

Advertencia: Locale 'es_CO.UTF-8' o 'es_ES.UTF-8' no encontrado. Usando locale por defecto del sistema para fechas.
Informe completo con portada creado en: /content/drive/MyDrive/EVALUACIÓN DOCENTE/Informe_Evaluacion_Syllabus.pdf
Proceso completado.


In [5]:
import pandas as pd
df = pd.read_excel("/content/drive/MyDrive/EVALUACIÓN DOCENTE/Seguimiento a Syllabus(5001-6425).xlsx")

# Diccionario de mapeo de etiquetas a valores
etiquetas_a_valores = {
    "Siempre": 5,
    "Muy a menudo": 4,
    "A menudo": 3,
    "Raras veces": 2,
    "Casi Nunca": 1
}

# Columnas objetivo (etiquetas de evaluación)
columnas_evaluacion = [
    "Las temáticas desarrolladas corresponden a las registradas en el syllabus.",
    "Las temáticas o contenidos se desarrollan acorde a los tiempos planeados en el syllabus",
    "Las actividades didácticas desarrolladas son acordes a la temática que se está abordado",
    "La estrategia didáctica es pertinente para el desarrollo de los resultados de aprendizaje que se pretende desarrollar.",
    "Los mecanismos (estrategias) de evaluación son pertinentes para comprobar los resultados de aprendizaje que se esperan desarrollar en el espacio académico.",
    "Se implementan diferentes metodologías de evaluación"
]

# Columnas de puntos correspondientes
columnas_puntos = [
    "Puntos: Las temáticas desarrolladas corresponden a las registradas en el syllabus.",
    "Puntos: Las temáticas o contenidos se desarrollan acorde a los tiempos planeados en el syllabus",
    "Puntos: Las actividades didácticas desarrolladas son acordes a la temática que se está abordado",
    "Puntos: La estrategia didáctica es pertinente para el desarrollo de los resultados de aprendizaje que se pretende desarrollar.",
    "Puntos: Los mecanismos (estrategias) de evaluación son pertinentes para comprobar los resultados de aprendizaje que se esperan desarrollar en el espacio académico.",
    "Puntos: Se implementan diferentes metodologías de evaluación"
]

# Aplicar el mapeo en cada columna de evaluación y asignar el resultado en la columna de puntos
for col_eval, col_punto in zip(columnas_evaluacion, columnas_puntos):
    df[col_punto] = df[col_eval].map(etiquetas_a_valores)

# Exportar a un nuevo archivo Excel
output_path = "/content/drive/MyDrive/EVALUACIÓN DOCENTE/seguimiento_ponderado.xlsx"
df.to_excel(output_path, index=False)

output_path

'/content/drive/MyDrive/EVALUACIÓN DOCENTE/seguimiento_ponderado.xlsx'